# Tutorial Expectation Suite
### Introduction
Welcome to the tutorial! This notebook will help you run through a full example of using Grater Expectations. To repeat on what is already mentioned in the README, note that you need the following to run all the components of the tutorial:

-  AWS account with [programmatic access keys](https://docs.aws.amazon.com/general/latest/gr/aws-sec-cred-types.html)
- [Docker Engine](https://docs.docker.com/engine/): to create new images to run on AWS Lambda and push them to ECR
- [AWS CLI](https://aws.amazon.com/cli/): to login to AWS, create an ECR repository and push docker images to ECR
- Python 3.8: It is recommended to use conda ([Miniconda](https://docs.conda.io/en/latest/miniconda.html)) for easy environment creation and management
- [Terraform](https://www.terraform.io/): to spin up S3 buckets for GE artifacts and the Data Docs website and a Lambda function for testing
- IDE (e.g. VS Code, optional): for easier development (not necessarily for notebooks, but definitely for Python files)

If you have these installed, then you are ready to continue with the tutorial!

<hr>

In order to validate your data, Great Expectations is a package that offers a battery-included set of logic to get up-and-running fast. Fully figuring out how Great Expectations works and applying it to your project, however, can be somewhat involved. This is what Grater Expectations and this tutorial help you with!

Grater Expectations makes a few choices for you and offers scripts, configurations and notebooks to get you started. The choices that were made are:

- Great Expectations output will be stored on S3
- The rendered Data Docs site will be stored on S3
- You will write your own data loading logic to read data into memory as a pandas DataFrame
- You will write your own set of expectations to test the quality of this data
- The validation logic will be deployed as Docker container via AWS Lambda

To set you up for the above, you already entered configurations in `testing_config.yml` for the tutorial, which will be used throughout the code to generate AWS services and access them.


### Description
This notebook will guide you through each of the steps to get you testing your data as soon as possible. The steps are:

1. TODO

<br>
<hr>


#### Setting up a virtual environment
In order to run the logic contained within this notebook, make sure that it was started up from a virtual environment that contained all required Python dependencies. The easiest way to assure this is to first make a virtual environment for the project and then call `initialize_project.py` from within it.

To create a new virtual environment, e.g. for python 3.8, and installing the packages that can be found in the requirements.txt file, the commands are (ran from the root directory of this repository):
```sh
conda create --name <virtual_environment_name> python=3.8
conda activate <virtual_environment_name>
pip install -r </path/to/requirements.txt>
```

Make sure that you use this virtual environment when you run the code in this notebook, but also when you are testing the Lambda function.


<br>
<hr>


#### Provisioning S3 buckets
For testing, Grater Expectations is configured to interact with S3 buckets on AWS. To be able to run all the code in this notebook the **store_bucket**, **site_bucket** and **data_bucket** as configured in the testing_config.yml must be provisioned.

To do so, auto-generated Terraform files can be found in the *terraform/buckets* directory of this project. To use these configurations to generate S3 buckets in that directory, open a (Git bash) terminal, set your AWS programmatic access credentials as environment variables and run the following commands:

```bash
# Set credentials
export AWS_ACCESS_KEY_ID=<enter_aws_access_key_here>
export AWS_SECRET_ACCESS_KEY=<enter_aws_secret_access_key_here>

# Go to correct Terraform directory
cd terraform/buckets

# Initialize Terraform
terraform init

# Generate deployment plan, enter yes at the prompt if the plan is correct
terraform apply
```

Terraform will then provide you with a plan, which it will deploy if you enter 'yes' at the prompt as shown below.

![Terraform bucket prompt](../docs/images/tutorial_tf_bucket_prompt.png)

After entering yes, Terraform will deploy the buckets and will show you the following output when finished.

![Terraform bucket apply](../docs/images/tutorial_tf_bucket_apply.png)

<br>
<hr>

#### Imports and configurations
In the cell below, packages are imported and configurations are loaded from the `project_config.yml`, which was automatically created by `initialize_project.py` and contains the configuration parameters for this tutorial.

Note specifically the import of load_csv_from_s3 as load_data. For this tutorial, a function was already developed to load csv files from S3 and turn these into pandas DataFrames. You can expect the code for this function in `supporting_functions.py` or by calling `help(load_data)`

In [ ]:
# Imports of Python and Grater Expectations packages and logic
import great_expectations as ge
from great_expectations.core.batch import RuntimeBatchRequest
import boto3
from supporting_functions import (TestingConfiguration,
                                  get_file_keys_from_s3,
                                  print_ge_site_link,
                                  generate_link_in_notebook,
                                  invoke_lambda_function)
from supporting_functions import load_csv_from_s3 as load_data
import json
import os

# -- Load parameters from configuration file
test_config = TestingConfiguration("project_config.yml")
test_config.load_config()

#### Initialization of objects
Next, objects required to interact with AWS from Python and a Great Expectations DataContext are initialized.

By loading the GE configuration file (found in the subdirectory great_expectations), the `DataContext` object stores important parameters for you to interact with GE and automatically interact with S3 buckets for storing output, pulling checkpoints and generating Data Docs sites. For more information on data contexts, check out [the documentation](https://docs.greatexpectations.io/docs/terms/data_context/)

In [ ]:
# -- 1. Initialize GE and S3 objects
s3_client = boto3.client("s3")
bucket = boto3.resource("s3").Bucket(test_config.data_bucket)
context = ge.data_context.DataContext()

#### Uploading data to S3

To emulate a *normal* setting, you will now upload the tutorial data found in this repository to the S3 data bucket you provisioned using terraform. In order to do so, the PATH_TUTORIAL_DATA constant is set to the path to the data. Next, the initialized bucket client (which is set to the data bucket) is used to upload the data to your bucket.

After running the code cell below, you can enter the AWS terminal and check the S3 bucket to see if the data now resides on it. The cell automatically provides a link to the bucket that you can click.

In [ ]:
# -- Path constant
PATH_TUTORIAL_DATA = "data/"

# -- Upload logic
for dataset in os.listdir(PATH_TUTORIAL_DATA):
    # -- Set path to file
    path_file = os.path.abspath(PATH_TUTORIAL_DATA+dataset)

    # -- Upload to S3
    bucket.meta.client.upload_file(
        Filename=path_file, 
        Bucket=test_config.data_bucket, 
        Key=PATH_TUTORIAL_DATA+dataset
    )

# -- Provide link to S3 bucket
generate_link_in_notebook(f"https://{bucket.name}.s3.amazonaws.com/")

#### Loading data

The next step is to load an example dataset that can be used to create expectations for. As previously mentioned, the function `load_csv_from_s3` was already developed for this tutorial and was imported as `load` data. 

Having uploaded the data to S3 in the previous cell, we can now download it and transform it into a pandas DataFrame with the cell below. To know which file to load, we first get a list of objects residing on S3 using `get_file_keys_from_s3` and then pick the oldest one from this list (objects are sorted).

The code of the `load_csv_from_s3` function is shown below.
<br>

```python
def load_csv_from_s3(
    s3_bucket_client: boto3.resource("s3").Bucket, prefix: str
) -> pd.DataFrame:
    """Function to loads a csv from S3 into a pandas DataFrame

    Parameters
    ----------
    s3_bucket_client : boto3.resource
        Instantiated s3 bucket client using boto3. Note that it should already
        be pointing to the bucket from which you want to load objects
    prefix : str
        Prefix to csv object on S3

    Returns
    -------
    pd.DataFrame
        The loaded csv object as pandas DataFrame
    """
    s3_object = s3_bucket_client.Object(prefix).get()
    df = pd.read_csv(s3_object["Body"])

    return df
```
<br>

Pay special attention to what inputs this function needs and how it knows what file to load, since this will be used in the Lambda function as well. As you can see in the function source code, it will need an initialized bucket client and a prefix to an object in order to load data.

Apart from the dataset, Great Expectations needs some additional parameters for operations down the line. These are an identifier for the batch being run and a name for the data asset. These can be the same, as long as they can be used to identify which dataset is being evaluated.

In [ ]:
# -- Get object keys from S3, sort, pick oldest
list_objects = get_file_keys_from_s3(s3_client, test_config.data_bucket, PATH_TUTORIAL_DATA)
list_objects.sort()
asset_name = list_objects[0]

# -- Load dataset
df_batch = load_data(bucket, asset_name)
batch_identifier = "tutorial_batch_dataset"

#### Generate batch request, generate suite and start validator
Now that the data has been loaded, this batch will be passed to a RuntimeBatchRequest below in order to start building an expectation suite. An expectation suite is Great Expectations jargon for a collection of expectations (or tests) that you want to run your data against. 

Great Expectations requires data to be passed as a request when you want to use the context object to generate things such as expectation suites. The RuntimeBatchRequest is used here, because we are loading data (with our own logic) at runtime and want to pass that to subsequent objects.

More information on runtime batch requests can be found [here](https://docs.greatexpectations.io/docs/guides/connecting_to_your_data/how_to_configure_a_runtimedataconnector/).

Using the RuntimeBatchRequest, two things are done next:
1. Generating an expectation suite: this will serve as a collection of tests you will run for future datasets
2. Generate a validator: this object will use the batch dataset loaded previously to start running expectations and storing these in the suite

As soon as the suite and validator are initiated, you can start writing expectations in the next cells

In [ ]:
# -- 3. Generate batch request at runtime using loaded tile
batch_request = RuntimeBatchRequest(
    datasource_name="runtime_data",
    data_connector_name="runtime_data_connector",
    data_asset_name=asset_name,
    runtime_parameters={"batch_data": df_batch},
    batch_identifiers={"batch_identifier": batch_identifier,},
)

# -- 4. Generate expectation suite, start validator
suite = context.create_expectation_suite(
    test_config.expectations_suite_name,
    overwrite_existing=True,  
)

validator = context.get_validator(
    batch_request=batch_request,
    expectation_suite_name=test_config.expectations_suite_name,
)

#### Expectations
Using the validator object, expectations can be formulated below. Since Great Expectations comes with many expectations out of the box, [this page](https://greatexpectations.io/expectations) is generally a good place to start browsing through these. 

Predefined expectations can be used by calling them using the validator object and passing the required arguments. For example, to run an expectation on the number of rows in a dataframe, the following snippet can be used:

```python
# Get number of rows of current batch
row_count = df_batch.shape[0]

# Make expectation where the maximum deviation from the batch number of rows is 1%
max_delta = 0.01
validator.expect_table_row_count_to_be_between(
    min_value=row_count * (1-max_delta), max_value=row_count * (1+max_delta))
```

Expectations can be run both on the level of a table, e.g. evaluating the number of rows or columns, and on the level of a column, e.g. evaluating the minimum and maximum within a column. The expected values for such tests are set when generating the expectations suite using the `validator` object in the cells below and will be used for future validations.

Alternatively, expectations can also be set using dynamic evaluation parameters, which is just an expensive set of words for test values that you determine at runtime. This can be useful if you for example want to compare your current dataset with the data of last month and use values in your expectations based on last month's data. An example of how to configure these dynamic evaluation parameters is shown below. More information about them can be found [here](https://docs.greatexpectations.io/docs/reference/evaluation_parameters/)

Apart from existing expectations, you can also develop expectations yourself. If you want to do so, more information can be found about that [here](https://docs.greatexpectations.io/docs/guides/expectations/creating_custom_expectations/overview)

In [ ]:
# -- Table level expectations

# -- Constant parameters
ROW_COUNT_DELTA = .05

# -- 1. Expect future datasets to contain the same columns as current batch DataFrame
expected_columns = df_batch.columns
validator.expect_table_columns_to_match_set(
    column_set=expected_columns, exact_match=True
)

# -- 2. Expect row count to be in between range, based on set delta and number of rows of batch dataset
row_count = df_batch.shape[0]
validator.expect_table_row_count_to_be_between(
    min_value=row_count * (1-ROW_COUNT_DELTA), 
    max_value=row_count * (1+ROW_COUNT_DELTA),
)

In [ ]:
# -- Column level expectations

# -- 1. Values are never null
for column in expected_columns:
    validator.expect_column_values_to_not_be_null(column)

# -- 2. Date columns are parseable
date_columns = ["tpep_pickup_datetime", "tpep_dropoff_datetime"]
for column in date_columns:
    validator.expect_column_values_to_be_dateutil_parseable(column)

# -- 3. Check dtypes, assuming dtypes of the batch dataset are correct (this is
#       something you might rather want to hard-code for real products)
dict_dtypes = {}
for column, dtype in zip(df_batch.dtypes.index, df_batch.dtypes):
    dict_dtypes[column] = str(dtype)

for column, dtype in dict_dtypes.items():
    validator.expect_column_values_to_be_of_type(column, dtype)

# -- 4. Expect values of specific columns to be between lower- and upper bounds
dict_bounds = {"VendorID":[1,2],
              "payment_type":[1,4]
              }

for column, (lower_bound, upper_bound) in dict_bounds.items():
    # NOTE: for large datasets, this expectation is really slow. Using seperate tests
    # for what the minimum should be and the maximum should be is a lot faster, while
    # achieving the same test-wise
    validator.expect_column_values_to_be_between(column, lower_bound, upper_bound)

In [ ]:
# -- Expectation using dynamic evaluation parameters. Instead of passing values directly 
#    here, a dictionary is passed with the name of the dynamic parameters along with a mock
#    value for the current validation. At runtime, this value will need to be provided 
#    when running the validations

test_column = "passenger_count"
max_passenger_count = df_batch[test_column].max()

validator.expect_column_max_to_be_between(
    column=test_column,
    min_value={
        "$PARAMETER": "min_max_passenger_count",
        "$PARAMETER.min_max_passenger_count": max_passenger_count
        },
    max_value={
        "$PARAMETER": "max_max_passenger_count",
        "$PARAMETER.max_max_passenger_count": max_passenger_count+2
        },
)

#### Finalize suite and create checkpoint
After running all the expectations in the cells above, the cell below can be executed to save this set of expectations as a suite and couple it with a checkpoint. 

A checkpoint is an object which can be called by validation logic to run new data batches against, coupling an expectation_suite with parameters to a name. In addition, actions can be coupled to this checkpoint, such as automatically updating the Data Docs website or sending a message on Slack. 

This checkpoint can be used in other scripts (lambda_function.py) by passing a new batch of data (as RuntimeBatchRequest) along with the checkpoint name to the `run_checkpoint` method of an initialized DataContext (which is the same as the `context` object initialized in this notebook). Such a call would look like:

```python
results = context.run_checkpoint(
    checkpoint_name="CHECKPOINT_NAME",
    validations=[{"batch_request": batch_request}],
    )
```

When the code below is ran, Great Expectations automatically saves the expectation suite and checkpoint to S3 via the validator and context objects.

In [ ]:
# -- 6. Save suite
validator.save_expectation_suite(discard_failed_expectations=False)

# -- 7.1 Create Simple checkpoint with automatic data docs updates
checkpoint_config = {
    "name": test_config.checkpoint_name,
    "config_version": 3,
    "class_name": "SimpleCheckpoint",
    "expectation_suite_name": test_config.expectations_suite_name,
    "run_name_template": test_config.run_name_template,
}

# -- 7.2 Add to context object
context.add_checkpoint(**checkpoint_config)

#### Instantiate Data Docs website
Next, the command below can be ran to build the Data Docs website on S3, which provides an interactive user interface in which you can browse through expectation suites and check validation results. More on Data Docs can be found [here](https://docs.greatexpectations.io/docs/reference/data_docs/)

If you use a SimpleCheckpoint, as is the case in this tutorial, the website will automatically be updated each time validaitons are run. If not, you have to either manually or programmatically update the Data Docs website but calling `context.build_data_docs()`

After initializing the Data Docs website, you should be able to see the expectation suite we just generated. In the following steps, we will start running validations, which well then also appear on the website.

In [ ]:
ge_site_output = context.build_data_docs()
print_ge_site_link(ge_site_output)

#### Develop logic for Lambda function

#### Create Docker Image and deploy on AWS Lambda

#### Next steps

After you have run all of the steps above, instantiated a Data Docs website and can see your expectation suite there, you are ready to proceed with developing the logic for the Lambda function in `lambda_function.py`, creating a Docker Image with that, uploading that to ECR and deploying it as a Lambda function.

Please refer to the README for more information.

#### Stop Jupyter server
After running all commands above and generating an expectation suite, checkpoint and website, run the command below to stop the Jupyter server from running

In [ ]:
os.system("jupyter notebook stop")